<a href="https://colab.research.google.com/github/e-olang/NLP/blob/main/Language%20Modeling/ann_lang_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [28]:
data_text = """The unanimous Declaration of the thirteen united States of America, When in the Course of human events, it becomes necessary for one people to dissolve the political bands which have connected them with another, and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature and of Nature's God entitle them, a decent respect to the opinions of mankind requires that they should declare the causes which impel them to the separation.

We hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness.--That to secure these rights, Governments are instituted among Men, deriving their just powers from the consent of the governed, --That whenever any Form of Government becomes destructive of these ends, it is the Right of the People to alter or to abolish it, and to institute new Government, laying its foundation on such principles and organizing its powers in such form, as to them shall seem most likely to effect their Safety and Happiness. Prudence, indeed, will dictate that Governments long established should not be changed for light and transient causes; and accordingly all experience hath shewn, that mankind are more disposed to suffer, while evils are sufferable, than to right themselves by abolishing the forms to which they are accustomed. But when a long train of abuses and usurpations, pursuing invariably the same Object evinces a design to reduce them under absolute Despotism, it is their right, it is their duty, to throw off such Government, and to provide new Guards for their future security.--Such has been the patient sufferance of these Colonies; and such is now the necessity which constrains them to alter their former Systems of Government. The history of the present King of Great Britain is a history of repeated injuries and usurpations, all having in direct object the establishment of an absolute Tyranny over these States. To prove this, let Facts be submitted to a candid world.

He has refused his Assent to Laws, the most wholesome and necessary for the public good.

He has forbidden his Governors to pass Laws of immediate and pressing importance, unless suspended in their operation till his Assent should be obtained; and when so suspended, he has utterly neglected to attend to them.

He has refused to pass other Laws for the accommodation of large districts of people, unless those people would relinquish the right of Representation in the Legislature, a right inestimable to them and formidable to tyrants only.

He has called together legislative bodies at places unusual, uncomfortable, and distant from the depository of their public Records, for the sole purpose of fatiguing them into compliance with his measures.

He has dissolved Representative Houses repeatedly, for opposing with manly firmness his invasions on the rights of the people.

He has refused for a long time, after such dissolutions, to cause others to be elected; whereby the Legislative powers, incapable of Annihilation, have returned to the People at large for their exercise; the State remaining in the mean time exposed to all the dangers of invasion from without, and convulsions within.

He has endeavoured to prevent the population of these States; for that purpose obstructing the Laws for Naturalization of Foreigners; refusing to pass others to encourage their migrations hither, and raising the conditions of new Appropriations of Lands.

He has obstructed the Administration of Justice, by refusing his Assent to Laws for establishing Judiciary powers.

He has made Judges dependent on his Will alone, for the tenure of their offices, and the amount and payment of their salaries.

He has erected a multitude of New Offices, and sent hither swarms of Officers to harrass our people, and eat out their substance.

He has kept among us, in times of peace, Standing Armies without the Consent of our legislatures.

He has affected to render the Military independent of and superior to the Civil power.

He has combined with others to subject us to a jurisdiction foreign to our constitution, and unacknowledged by our laws; giving his Assent to their Acts of pretended Legislation:

For Quartering large bodies of armed troops among us:

For protecting them, by a mock Trial, from punishment for any Murders which they should commit on the Inhabitants of these States:

For cutting off our Trade with all parts of the world:

For imposing Taxes on us without our Consent:

For depriving us in many cases, of the benefits of Trial by Jury:

For transporting us beyond Seas to be tried for pretended offences

For abolishing the free System of English Laws in a neighbouring Province, establishing therein an Arbitrary government, and enlarging its Boundaries so as to render it at once an example and fit instrument for introducing the same absolute rule into these Colonies:

For taking away our Charters, abolishing our most valuable Laws, and altering fundamentally the Forms of our Governments:

For suspending our own Legislatures, and declaring themselves invested with power to legislate for us in all cases whatsoever.

He has abdicated Government here, by declaring us out of his Protection and waging War against us.

He has plundered our seas, ravaged our Coasts, burnt our towns, and destroyed the lives of our people.

He is at this time transporting large Armies of foreign Mercenaries to compleat the works of death, desolation and tyranny, already begun with circumstances of Cruelty & perfidy scarcely paralleled in the most barbarous ages, and totally unworthy the Head of a civilized nation.

He has constrained our fellow Citizens taken Captive on the high Seas to bear Arms against their Country, to become the executioners of their friends and Brethren, or to fall themselves by their Hands.

He has excited domestic insurrections amongst us, and has endeavoured to bring on the inhabitants of our frontiers, the merciless Indian Savages, whose known rule of warfare, is an undistinguished destruction of all ages, sexes and conditions.

In every stage of these Oppressions We have Petitioned for Redress in the most humble terms: Our repeated Petitions have been answered only by repeated injury. A Prince whose character is thus marked by every act which may define a Tyrant, is unfit to be the ruler of a free people.

Nor have We been wanting in attentions to our Brittish brethren. We have warned them from time to time of attempts by their legislature to extend an unwarrantable jurisdiction over us. We have reminded them of the circumstances of our emigration and settlement here. We have appealed to their native justice and magnanimity, and we have conjured them by the ties of our common kindred to disavow these usurpations, which, would inevitably interrupt our connections and correspondence. They too have been deaf to the voice of justice and of consanguinity. We must, therefore, acquiesce in the necessity, which denounces our Separation, and hold them, as we hold the rest of mankind, Enemies in War, in Peace Friends.

We, therefore, the Representatives of the united States of America, in General Congress, Assembled, appealing to the Supreme Judge of the world for the rectitude of our intentions, do, in the Name, and by Authority of the good People of these Colonies, solemnly publish and declare, That these United Colonies are, and of Right ought to be Free and Independent States; that they are Absolved from all Allegiance to the British Crown, and that all political connection between them and the State of Great Britain, is and ought to be totally dissolved; and that as Free and Independent States, they have full Power to levy War, conclude Peace, contract Alliances, establish Commerce, and to do all other Acts and Things which Independent States may of right do. And for the support of this Declaration, with a firm reliance on the protection of divine Providence, we mutually pledge to each other our Lives, our Fortunes and our sacred Honor."""

In [31]:
import re

def cleaner(text):
    # lower case
    newtext = text.lower()
    newtext = re.sub(r"'s\b","",newtext)

    # remove punctuation
    newtext = re.sub("[^a-zA-Z]", " ", newtext)
    long_words = []

    # remove short words
    for i in newtext.split():
        if len(i) >= 3:
            long_words.append(i)
    return (" ".join(long_words)).strip()

data_new = cleaner(data_text)

In [32]:
data_new

'the unanimous declaration the thirteen united states america when the course human events becomes necessary for one people dissolve the political bands which have connected them with another and assume among the powers the earth the separate and equal station which the laws nature and nature god entitle them decent respect the opinions mankind requires that they should declare the causes which impel them the separation hold these truths self evident that all men are created equal that they are endowed their creator with certain unalienable rights that among these are life liberty and the pursuit happiness that secure these rights governments are instituted among men deriving their just powers from the consent the governed that whenever any form government becomes destructive these ends the right the people alter abolish and institute new government laying its foundation such principles and organizing its powers such form them shall seem most likely effect their safety and happiness pr

In [35]:
def create_seq(text):
    length = 30
    sequences = list()

    for i in range(length, len(text)):
        # select seq of tokens
        seq = text[i - length: i + 1]
        
        # store
        sequences.append(seq)

    print('Total Sequences: %d' %len(sequences))
    return sequences

# create seq.
sequences = create_seq(data_new)



Total Sequences: 7052


In [37]:
# encoding seq

chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        # int encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)

    return sequences

# encode the seqs.
sequences = encode_seq(sequences)

In [40]:
from sklearn.model_selection import train_test_split

# vocab size
vocab = len(mapping)
sequences = np.array(sequences)

# create X and y
X, y = sequences[:, :-1], sequences[:, -1]

# one hot encode y
y = to_categorical(y, num_classes = vocab)

# create train and valid sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size= 0.1, random_state = 42)

print('Train shape:', X_tr.shape, 'Val shape', X_val.shape)

Train shape: (6346, 30) Val shape (706, 30)


In [42]:
# model

model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable = True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))

print(model.summary())

model.compile(loss = 'categorical_crossentropy', metrics = ['acc'], optimizer = 'adam')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 30, 50)            1350      
                                                                 
 gru_1 (GRU)                 (None, 150)               90900     
                                                                 
 dense_1 (Dense)             (None, 27)                4077      
                                                                 
Total params: 96,327
Trainable params: 96,327
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
model.fit(X_tr, y_tr, epochs = 100, verbose = 2, validation_data = (X_val, y_val))

Epoch 1/100
199/199 - 20s - loss: 2.7464 - acc: 0.2067 - val_loss: 2.3961 - val_acc: 0.2989 - 20s/epoch - 101ms/step
Epoch 2/100
199/199 - 17s - loss: 2.2967 - acc: 0.3256 - val_loss: 2.2295 - val_acc: 0.3329 - 17s/epoch - 84ms/step
Epoch 3/100
199/199 - 17s - loss: 2.1647 - acc: 0.3569 - val_loss: 2.1428 - val_acc: 0.3768 - 17s/epoch - 84ms/step
Epoch 4/100
199/199 - 17s - loss: 2.0625 - acc: 0.3840 - val_loss: 2.0860 - val_acc: 0.3839 - 17s/epoch - 83ms/step
Epoch 5/100
199/199 - 17s - loss: 1.9800 - acc: 0.4092 - val_loss: 2.0456 - val_acc: 0.4008 - 17s/epoch - 83ms/step
Epoch 6/100
199/199 - 17s - loss: 1.8948 - acc: 0.4354 - val_loss: 2.0058 - val_acc: 0.4164 - 17s/epoch - 83ms/step
Epoch 7/100
199/199 - 17s - loss: 1.8218 - acc: 0.4527 - val_loss: 1.9673 - val_acc: 0.4433 - 17s/epoch - 83ms/step
Epoch 8/100
199/199 - 17s - loss: 1.7421 - acc: 0.4754 - val_loss: 1.9260 - val_acc: 0.4462 - 17s/epoch - 83ms/step
Epoch 9/100
199/199 - 17s - loss: 1.6781 - acc: 0.4973 - val_loss: 1.90

In [62]:
# generating sequences

def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text

    # generate a fixed number of chars
    for _ in range(n_chars):
        # encode the chars as integers
        encoded = [mapping[char] for char in in_text]

        # truncate sequences to fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating = 'pre')

        # predict character
        yhat = model.predict(encoded, verbose = 0)

        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break

        # append to input
        in_text += char
    
    return in_text

In [63]:
inp = 'large groups'
print(len(inp))
print(generate_seq(model, mapping, 30, inp.lower(), 15))

12


ValueError: ignored

-----

In [45]:
model.save('model1.h5')

In [46]:
!sudo apt-get install graphviz && pip3 install graphviz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.40.1-2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [51]:
!pip3 install ann_visualizer --quiet

In [53]:
from ann_visualizer.visualize import ann_viz;
#Build your model here
ann_viz(model)

ValueError: ignored